In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = "mps" if torch.mps.is_available() else "cpu"
print(device)

block_size = 8
batch_size = 4

eval_iter = 250
max_iters = 10000
learning_rate = 3e-4
dropout = 0.2

mps


In [2]:
with open ("wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [3]:
string_to_int = {ch:i for i, ch in enumerate(chars) }
int_to_string = {i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

In [4]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print("Inputs")
print(x)
print("target")
print(y)

Inputs
tensor([[38, 68, 76,  1, 78, 68, 74,  1],
        [ 1,  1, 21,  1,  1, 44, 32, 29],
        [ 1, 65, 58, 59, 73,  1, 73, 61],
        [60,  1, 54, 60, 54, 62, 67, 72]], device='mps:0')
target
tensor([[68, 76,  1, 78, 68, 74,  1, 66],
        [ 1, 21,  1,  1, 44, 32, 29, 49],
        [65, 58, 59, 73,  1, 73, 61, 58],
        [ 1, 54, 60, 54, 62, 67, 72, 73]], device='mps:0')


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()

    for split in ['train', 'val']:
        losses = torch.zeros(eval_iter)
        for k in range(eval_iter):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embeddings_table(index)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
    
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, index, max_new_tokens):
        for i in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


;U!GM5'!A5c;HxoyTEGEGugdLP'l﻿)﻿)4I;HSG [1'3﻿g"bqE6)Fz;f
8pdLWpPJ5S6z fFbOTNga1*nDbF"?r5.Y6gpuXa1*blDc)ae-4CrV;&40'7dk",vS'ws4qR)SmpZzIxO﻿9gfhC)Gh",G)cnLAVW6&﻿g0o?
bm],i*8_WFL&B&A3ktOrA(,padQ Cp-KlUpEWO52nLi
e2XLX7GR(Qm.:Q1nbo",v5doF3.46WFS9_)8_8ovg0oiz]xnp-MMC!JXNMw?CkkPMww2n,A*xi*DKt
E4g3spdA68Zc m(9fsKN00DbmGdPZ?xoiz,AcV.ex-gzXov?j.eoVbeY1kI?SBEa;U3(rusMcc8﻿Dbi3!kpaFN﻿)r,9U﻿IE﻿tH"fr"*?jFKnp&"sK[ U6[7X)0kQ1tch&",(yuio﻿TG1ou3-?32TZL6(g1s&zmWQEMs3sMCXk"_Fdf;6m8wi
;7B)CXniouwH1u kjmal)ZchNkUa:OSAV


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iter == 0:
        losses = estimate_loss()
        print(f"split: {iter}, train loss: {losses['train']:.4f}, val: {losses['val']}")
    
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

split: 0, train loss: 4.8462, val: 4.850250720977783
split: 250, train loss: 4.7599, val: 4.7786335945129395
split: 500, train loss: 4.6945, val: 4.729936599731445
split: 750, train loss: 4.6509, val: 4.674564361572266
split: 1000, train loss: 4.5964, val: 4.604599475860596
split: 1250, train loss: 4.5342, val: 4.556427955627441
split: 1500, train loss: 4.5004, val: 4.479836940765381
split: 1750, train loss: 4.4247, val: 4.463559150695801
split: 2000, train loss: 4.3664, val: 4.381195545196533
split: 2250, train loss: 4.3122, val: 4.313987731933594
split: 2500, train loss: 4.2348, val: 4.271714687347412
split: 2750, train loss: 4.2219, val: 4.213782787322998
split: 3000, train loss: 4.1534, val: 4.163388729095459
split: 3250, train loss: 4.1102, val: 4.114892482757568
split: 3500, train loss: 4.0542, val: 4.047268867492676
split: 3750, train loss: 4.0068, val: 4.0339155197143555
split: 4000, train loss: 3.9586, val: 3.974841833114624
split: 4250, train loss: 3.9239, val: 3.921863794326

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)

In [11]:
context = torch.zeros((1, 1), dtype=torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"Othlanuthenjt hellllivonervoW0H7Wvice?FLepooousl30HS
wimindlthem aththo, bbs.); m.4﻿he'man by ave soulR irond
ckn.JEGXowhey a  aupefuiraffI_  bue hcosakitiouinLVsca k-QK﻿5Fwila  vedl,dyoolould:8;YU_7grm forse,"R8)RTie s d thunB
porsor-pangcit J-adre EYt OBV"hanunx]Btflm fup!-he?E2Yo!ERus acewh, q8hewowisaxp[wlmp0wir6Ggirit7L2. w 1K.]Ifo e rison
su Esm,
tyh
"M0!ee thl  be c[XFk*Cbedrifofor he sC"E"
ylap:uild, tyas wot an.cht imy. sit asso tis ZIthaulwFmorit ader'DTAryliigay7W?-D4﻿be mperildsoe w
